In [2]:
# ================================================================
# Import Dependencies
# ---------------------------------------------------------------
# This section loads all external libraries required for:
#   • Data manipulation and numerical computation (pandas, numpy)
#   • Flight trajectory parsing, visualization, and aeronautical data (traffic)
#   • Aircraft performance models (OpenAP)
#   • Fuel estimation using ACROPOLE models
#   • Miscellaneous utilities such as file paths and display tools
#
# Notes:
# - Warnings are intentionally suppressed for cleaner notebook output.
# - All imported modules are used in downstream processing of flight
#   trajectories,feature generation, and fuel estimation.
# ================================================================

from pathlib import Path                     # Cross-platform filesystem path handling

import pandas as pd                          # Primary data manipulation library
import numpy as np                           # Numerical computing and array utilities

# ---------------------- Traffic Library -------------------------
# traffic.core.Traffic  -> Handles flight trajectories and ADS-B/Mode-S time series
# traffic.data.airports -> Provides airport metadata (lat/lon, elevation, etc.)
# traffic.data.navaids  -> Provides navigational aids (VOR, NDB, FIX)
from traffic.core import Traffic
from traffic.data import airports as t_airports
from traffic.data import navaids

# ---------------------- OpenAP Models ---------------------------
# prop          -> Aircraft properties (mass, wingspan, reference area, etc.)
# FlightPhase   -> Automated flight-phase detection (CL, CR, DE, LVL, NA)
# Drag          -> Drag model: aerodynamic drag estimates
# Thrust        -> Engine thrust models
# FuelFlow      -> Fuel burn estimation based on power settings and flight conditions
from openap import prop
from openap.phase import FlightPhase
from openap.drag import Drag
from openap.thrust import Thrust
from openap import FuelFlow

# ---------------------- ACROPOLE Fuel Model ---------------------
# Advanced, calibrated fuel estimation tool for flight segments
from acropole import FuelEstimator

import math                                   # Mathematical utilities

# ---------------------- Warning Suppression ---------------------
# Suppress RuntimeWarning and UserWarning to avoid clutter during long computations.
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=UserWarning)


c:\Users\USER\miniconda3\envs\acropole\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\USER\miniconda3\envs\acropole\Lib\site-packages\acropole\estimator.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
# ---------------------------------------------------------------
# Pandas display configuration
# ---------------------------------------------------------------
# Ensure that all columns of any DataFrame are shown when displayed.
# This prevents column truncation, which is especially useful for
# inspecting wide flight-segment datasets with many engineered features.
pd.set_option('display.max_columns', None)


In [4]:
# ================================================================
# Dataset and Resource Paths
# ---------------------------------------------------------------
# Centralized definitions of all dataset locations used throughout
# the notebook. These paths point to:
#   • Flight trajectory folders for training, ranking, and final sets
#   • Flight metadata (flightlists)
#   • Fuel measurement and submission files
#   • Airport reference datasets
#   • FAA aircraft performance and characteristics
#
# Storing paths in named constants improves maintainability and
# makes it easier to switch between environments or folder
# structures without editing downstream code.
# ================================================================

TRAIN_FLIGHTS_DIR   = '../prc-2025-datasets/flights_train'          # Raw trajectory files for training flights
RANK_FLIGHTS_DIR    = '../prc-2025-datasets/flights_rank'           # Trajectory files for phase 1 ranking
FINAL_FLIGHTS_DIR   = '../prc-2025-datasets/flights_final'          # Trajectory files for final submission (phase 2)

FLIGHTLIST_TRAIN    = '../prc-2025-datasets/flightlist_train.parquet' # Metadata for training flights
FLIGHTLIST_RANK     = '../prc-2025-datasets/flightlist_rank.parquet'  # Metadata for ranking flights
FLIGHTLIST_FINAL    = '../prc-2025-datasets/flightlist_final.parquet' # Metadata for final submission flights

FUEL_RANK           = '../prc-2025-datasets/fuel_rank_submission.parquet' # output template for phase 1 predictions
FUEL_TRAIN          = '../prc-2025-datasets/fuel_train.parquet'           # Ground-truth fuel for model training
FUEL_FINAL          = '../prc-2025-datasets/fuel_final_submission.parquet' # Output template for final predictions

AIRPORTS            = '../prc-2025-datasets/apt.parquet'              # Provided airport reference dataset
WORLD_AIRPORTS      = '../World_Airports.csv'                         # External global airport database
FAA_AIRCRAFT_DATA   = '../aircraft_data.xlsx'                         # FAA aircraft specs to augment available features


In [5]:
# ---------------------------------------------------------------
# Convert directory strings to Path objects
# ---------------------------------------------------------------
# Converting dataset directory paths from raw strings to pathlib.Path
# objects provides several advantages:
#   • OS-independent path handling (Windows, Linux, macOS)
#   • Easier path concatenation using "/" operator
# This makes downstream code more readable and robust.
# ---------------------------------------------------------------

TRAIN_FLIGHTS_DIR = Path(TRAIN_FLIGHTS_DIR)
RANK_FLIGHTS_DIR  = Path(RANK_FLIGHTS_DIR)
FINAL_FLIGHTS_DIR = Path(FINAL_FLIGHTS_DIR)


In [6]:
# ================================================================
# Load Core Datasets
# ---------------------------------------------------------------
# This section reads all required parquet files into memory.
# These datasets include:
#   • Flight metadata for training, ranking, and final evaluation
#   • Airport reference data
#   • Fuel measurements for training, as well as the
#     submission template for the ranking and final phases.
#
# ================================================================

train_flightlist = pd.read_parquet(FLIGHTLIST_TRAIN)   # Flight metadata (training set)
rank_flightlist  = pd.read_parquet(FLIGHTLIST_RANK)    # Flight metadata (ranking submission set)
final_flightlist = pd.read_parquet(FLIGHTLIST_FINAL)   # Flight metadata (final submission set)

airports = pd.read_parquet(AIRPORTS)                   # Airport reference information

train_fuel = pd.read_parquet(FUEL_TRAIN)               # Ground-truth fuel for training
rank_fuel  = pd.read_parquet(FUEL_RANK)                # Template for phase 1 fuel predictions
final_fuel = pd.read_parquet(FUEL_FINAL)               # Template for final fuel predictions


In [7]:
# ---------------------------------------------------------------
# Collect unique aircraft types across all datasets
# ---------------------------------------------------------------
# The PRC dataset is split into training, ranking, and final sets.
# To ensure complete coverage when building aircraft-specific models
# or loading aircraft performance data (e.g., OpenAP, FAA specs),
# we compile a unified list of all aircraft types appearing in:
#   • train_flightlist
#   • rank_flightlist
#   • final_flightlist
#
# This guarantees that downstream preprocessing and feature
# engineering can handle every aircraft type present in the competition.
# ---------------------------------------------------------------

all_aircraft_types = (
    pd.concat([rank_flightlist, final_flightlist, train_flightlist])
      ['aircraft_type']
      .unique()
)


In [8]:
# ================================================================
# Remove Known Problematic Flights from the Training Fuel Dataset
# ---------------------------------------------------------------
# Several flights in the provided dataset contain invalid or
# inconsistent trajectory or fuel data. These abnormalities would
# degrade model training, so they are explicitly excluded.
#
# Summary of removed flights:
#
# 1. prc790175280  
#    - Contains a segment lasting ~39 minutes with only start/end
#      trajectory points.
#    - Altitude barely changes (33998 → 33997 ft), yet fuel
#      consumption exceeds 30,000 kg for a B744 — physically unrealistic.
#
# 2. prc777924876
#    - Early timestamps show extremely erratic position data.
#    - Trajectory begins at unrealistic altitudes prior to recorded
#      takeoff.
#    - Training fuel targets exist only for this invalid pre–takeoff
#      region.
#
# 3. prc799064748
#    - Total recorded flight time between KCOS and KDEN is ~20 minutes,
#      which is operationally impossible for this route and aircraft.
#
# 4. prc776443417 and prc776502511
#    - These two IDs refer to the *same flight*, each containing
#      complementary portions of the trajectory.
#    - Resolution:
#         • Keep prc776443417 in the training labels
#         • Remove prc776502511 from train_fuel
#         • Later, when generating training trajectories,
#           load both trajectories and merge them under ID prc776443417
#
# 5. prc778592434, prc778582087, prc778566666
#    - These three are also duplicate/complementary trajectories.
#    - Resolution:
#         • Keep prc778592434
#         • Remove prc778582087 and prc778566666 from train_fuel
#         • Later concatenate their trajectories under the surviving ID
#
# Only the fuel labels are filtered here; trajectory merging is handled
# during feature preparation.
# ================================================================

elim_flights = [
    'prc776502511',
    'prc778582087',
    'prc778566666',
    'prc790175280',
    'prc799064748',
    'prc777924876',
]

# Remove problematic flight IDs from the training fuel dataset
train_fuel = (
    train_fuel[~train_fuel['flight_id'].isin(elim_flights)]
    .reset_index(drop=True)
)


In [9]:
# ================================================================
# Supplement Missing Airport Elevation Data
# ---------------------------------------------------------------
# The provided airport dataset (apt.parquet) contains missing values
# in the `elevation` field for some ICAO airports. Accurate elevation
# is important for:
#   • determining pressure altitude
#   • computing atmospheric properties
#   • estimating takeoff/landing fuel usage
#
# To correct this, we supplement missing elevations using an external
# global airport dataset:
#   Source: Kaggle — "World Airports"
#   URL: https://kaggle.com/datasets/mexwell/world-airports
#
# Steps:
#   1. Identify airports with missing elevation in the provided dataset.
#   2. Load the external dataset.
#   3. Filter out any airports that still lack elevation in the external data.
#   4. Keep only those airports whose ICAO codes match the missing entries.
#   5. Extract a mapping {ICAO -> elevation_ft} for patching.
#
# The resulting dictionary `supplement_airport_elev` will be used later
# to fill missing elevations in the main airport table.
# ================================================================

# Identify airports missing elevation in the competition dataset
missing_airport_elev = airports[airports['elevation'].isna()]['icao'].unique()

# Load external world airport database (contains elevation_ft)
world_airport = pd.read_csv(WORLD_AIRPORTS)

# Keep only rows with valid elevation data
world_airport_no_missing_elev = world_airport[~world_airport['elevation_ft'].isna()]

# Filter to only the airports whose elevation is missing in our dataset
world_airport_no_missing_elev = world_airport_no_missing_elev[
    world_airport_no_missing_elev['airport_ident'].isin(missing_airport_elev)
]

# Reindex so ICAO codes act as keys
world_airport_no_missing_elev.index = world_airport_no_missing_elev['airport_ident']

# Convert to dictionary for easy lookup during patching
supplement_airport_elev = world_airport_no_missing_elev['elevation_ft'].to_dict()


In [10]:
# ================================================================
# Complete Missing Airport Elevations Using Traffic Library Sources
# ---------------------------------------------------------------
# After supplementing missing airport elevations using the external
# World Airports dataset, some ICAO codes may still not have a value.
# For these remaining airports, we attempt two fallback strategies:
#
#   1. traffic.data.airports (t_airports)
#        • Provides airport metadata, including altitude (in feet or meters
#          depending on source).
#        • Some airports in the competition dataset may be found here even
#          if not present in the external Kaggle dataset.
#
#   2. traffic.data.navaids (navaids)
#        • As a last resort, use the altitude of a nearby navaid sharing
#          the same ICAO identifier (rare, but can exist).
#
# Any airport for which both lookups fail remains missing and will be
# handled later (typically ignored or filled with model-dependent defaults).
# ================================================================

for mis_air in missing_airport_elev:
    if mis_air not in supplement_airport_elev:
        # Attempt to retrieve elevation from traffic's airport database
        try:
            supplement_airport_elev[mis_air] = t_airports[mis_air].altitude
        except Exception:
            # Fallback: try retrieving altitude from a matching navaid
            try:
                supplement_airport_elev[mis_air] = navaids.get(mis_air).altitude
            except Exception:
                # Leave missing if no reliable source exists
                pass


In [11]:
# ================================================================
# Manually supplement elevation for ZGOW (source: https://acukwik.com/Airport-Info/ZGOW)
#     Altitude = 51 ft
# ================================================================

supplement_airport_elev['ZGOW'] = 51


In [12]:
# ================================================================
# Fill missing airport elevations in the main dataset
# ---------------------------------------------------------------
# Using the dictionary `supplement_airport_elev` built from:
#   • World Airports dataset
#   • Traffic library (airports & navaids)
#   • Manual supplements
# we fill the missing `elevation` values in the main airports DataFrame.
#
# Logic:
#   • If `elevation` is already present, keep it.
#   • If missing, look up the ICAO code in `supplement_airport_elev`.
# ================================================================

airports['elevation'] = np.where(
    airports['elevation'].isna(),
    airports['icao'].map(supplement_airport_elev),
    airports['elevation']
)


In [13]:
airports['elevation'].isna().sum()

np.int64(198)

In [14]:
# ================================================================
# Enrich Training Flight Metadata with Airport Coordinates & Elevations
# ---------------------------------------------------------------
# To compute features related to flight performance (e.g., takeoff/landing
# weight, climb profiles), we augment each flight with its origin and
# destination airport data:
#   • Latitude
#   • Longitude
#   • Elevation
#
# Steps:
# 1. Merge `train_flightlist` with `airports` on the origin ICAO code.
#    - Columns renamed to `orig_lat`, `orig_long`, `orig_elev`.
#    - Original `icao` column dropped after merge.
# 2. Merge `train_flightlist` with `airports` on the destination ICAO code.
#    - Columns renamed to `dest_lat`, `dest_long`, `dest_elev`.
#    - Original `icao` column dropped after merge.
# 3. Extract flight takeoff and landing dates from the timestamp columns.
#    - Adds `takeoff_date` and `land_date` for daily-level analyses or
#      grouping purposes.
# ================================================================

# Merge origin airport information
train_flightlist = pd.merge(
    train_flightlist,
    airports,
    left_on='origin_icao',
    right_on='icao',
    how='left'
)
train_flightlist = train_flightlist.rename(
    columns={'longitude':'orig_long', 'latitude':'orig_lat', 'elevation':'orig_elev'}
).drop(['icao'], axis=1)

# Merge destination airport information
train_flightlist = pd.merge(
    train_flightlist,
    airports,
    left_on='destination_icao',
    right_on='icao',
    how='left'
)
train_flightlist = train_flightlist.rename(
    columns={'longitude':'dest_long', 'latitude':'dest_lat', 'elevation':'dest_elev'}
).drop(['icao'], axis=1)

# Extract takeoff and landing dates for potential daily aggregations
train_flightlist['takeoff_date'] = train_flightlist['takeoff'].dt.date
train_flightlist['land_date'] = train_flightlist['landed'].dt.date


In [15]:
# ================================================================
# Enrich Final Flight Metadata with Airport Coordinates & Elevations
# ---------------------------------------------------------------
# Similar to the training set, we augment each flight in the final
# submission set with origin and destination airport information:
#   • Latitude
#   • Longitude
#   • Elevation
#
# This ensures consistency in feature engineering across training
# and submission datasets.
#
# Steps:
# 1. Merge `final_flightlist` with `airports` on the origin ICAO code.
#    - Rename columns to `orig_lat`, `orig_long`, `orig_elev`.
#    - Drop redundant `icao` column.
# 2. Merge `final_flightlist` with `airports` on the destination ICAO code.
#    - Rename columns to `dest_lat`, `dest_long`, `dest_elev`.
#    - Drop redundant `icao` column.
# 3. Extract takeoff and landing dates for daily-level analyses or
#    feature creation.
# ================================================================

# Merge origin airport information
final_flightlist = pd.merge(
    final_flightlist,
    airports,
    left_on='origin_icao',
    right_on='icao',
    how='left'
)
final_flightlist = final_flightlist.rename(
    columns={'longitude':'orig_long', 'latitude':'orig_lat', 'elevation':'orig_elev'}
).drop(['icao'], axis=1)

# Merge destination airport information
final_flightlist = pd.merge(
    final_flightlist,
    airports,
    left_on='destination_icao',
    right_on='icao',
    how='left'
)
final_flightlist = final_flightlist.rename(
    columns={'longitude':'dest_long', 'latitude':'dest_lat', 'elevation':'dest_elev'}
).drop(['icao'], axis=1)

# Extract takeoff and landing dates for potential daily aggregation or feature creation
final_flightlist['takeoff_date'] = final_flightlist['takeoff'].dt.date
final_flightlist['land_date'] = final_flightlist['landed'].dt.date


In [16]:
# ================================================================
# Enrich Ranking Flight Metadata with Airport Coordinates & Elevations
# ---------------------------------------------------------------
# To maintain consistency with training and final datasets, we augment
# each flight in the ranking/validation set with origin and destination
# airport information:
#   • Latitude
#   • Longitude
#   • Elevation
#
# This ensures that all feature engineering steps can be applied
# uniformly across training, ranking, and final datasets.
#
# Steps:
# 1. Merge `rank_flightlist` with `airports` on the origin ICAO code.
#    - Rename columns to `orig_lat`, `orig_long`, `orig_elev`.
#    - Drop the redundant `icao` column.
# 2. Merge `rank_flightlist` with `airports` on the destination ICAO code.
#    - Rename columns to `dest_lat`, `dest_long`, `dest_elev`.
#    - Drop the redundant `icao` column.
# 3. Extract takeoff and landing dates for daily-level feature creation.
# ================================================================

# Merge origin airport information
rank_flightlist = pd.merge(
    rank_flightlist,
    airports,
    left_on='origin_icao',
    right_on='icao',
    how='left'
)
rank_flightlist = rank_flightlist.rename(
    columns={'longitude':'orig_long', 'latitude':'orig_lat', 'elevation':'orig_elev'}
).drop(['icao'], axis=1)

# Merge destination airport information
rank_flightlist = pd.merge(
    rank_flightlist,
    airports,
    left_on='destination_icao',
    right_on='icao',
    how='left'
)
rank_flightlist = rank_flightlist.rename(
    columns={'longitude':'dest_long', 'latitude':'dest_lat', 'elevation':'dest_elev'}
).drop(['icao'], axis=1)

# Extract takeoff and landing dates
rank_flightlist['takeoff_date'] = rank_flightlist['takeoff'].dt.date
rank_flightlist['land_date'] = rank_flightlist['landed'].dt.date


In [17]:
# ================================================================
# Define Aircraft Type Specifications
# ---------------------------------------------------------------
# This dictionary contains detailed information about the aircraft
# types appearing in the dataset. Information includes:
#   • Model identifiers and ICAO codes
#   • Aircraft class (L: large), wake turbulence category
#   • Engine type, number, and optional manufacturer/model/thrust
#   • Physical dimensions: length, wingspan, tail height
#   • Performance metrics: range (nm), ceiling (ft), max speed, passenger capacity
#
# Source references:
#   • 'https://github.com/atoff/OpenAircraftType/blob/master/src'
#
# Steps:
# 1. Define `aircraft_type_dict` mapping ICAO codes to aircraft specifications.
# 2. Convert dictionary to a pandas DataFrame for easier access and merging.
# 3. Rename `ICAO` column to `aircraft_type` to match flight datasets.
# ================================================================

# Dictionary mapping ICAO codes to aircraft specifications

aircraft_type_dict = {
    'B789':{'MODEL':'787-9 Dreamliner','ICAO':'B789','CLASS':'L','WAKE':'H','ENG_TYPE':'J',
            'ENG_NUM':2},
    'A359':{'MODEL':'A350-900','ICAO':'A359','CLASS':'L','WAKE':'H','ENG_TYPE':'J',
            'ENG_NUM':2,'ENG_NAME':'RR Trent XWB','ENG_MAN':'Rolls-Royce','ENG_MODEL':'Trent XWB',
            'ENG_THRUST':374500,'LENGTH':66.8,'WINGSPAN':64.75,'TAIL_HEIGHT':17.05,'RANGE':8100,
            'CEILING':43100,'MAX_SPEED':13,'PAX_CAP':325},
    'B788':{'MODEL':'787-8 Dreamliner','ICAO':'B788','CLASS':'L','WAKE':'H','ENG_TYPE':'J','ENG_NUM':2},
    'A332':{'MODEL':'A330-200','ICAO':'A332','CLASS':'L','WAKE':'H','ENG_TYPE':'J','ENG_NUM':2,
            'ENG_NAME':'GE CF6-80E/PW4000/RR Trent 700',
            'ENG_MAN':'General Electric/Pratt & Whitney/Rolls-Royce',
            'ENG_MODEL':'CF6-80E/PW4000/Trent 700','ENG_THRUST':316000,'LENGTH':58.82,
            'WINGSPAN':60.3,'TAIL_HEIGHT':17.39,'RANGE':7250,'CEILING':41100,'MAX_SPEED':70,
            'PAX_CAP':406},
    'A21N':{'MODEL':'A321neo','ICAO':'A21N','CLASS':'L','WAKE':'M','ENG_TYPE':'J','ENG_NUM':2,
            'ENG_NAME':'PW1000G/CFM LEAP-1A','ENG_MAN':'Pratt & Whitney/CFM International',
            'ENG_MODEL':'PW1000G/CFM LEAP-1A','ENG_THRUST':147300,'LENGTH':44.51,'WINGSPAN':35.8,
            'RANGE':4000,'CEILING':39800,'MAX_SPEED':473,'PAX_CAP':240},
    'A20N':{'MODEL':'A320neo','ICAO':'A20N','CLASS':'L','WAKE':'M','ENG_TYPE':'J','ENG_NUM':2,
            'ENG_NAME':'PW1000G/CFM LEAP-1A','ENG_MAN':'Pratt & Whitney/CFM International',
            'ENG_MODEL':'PW1000G/CFM LEAP-1A','ENG_THRUST':120600,'LENGTH':37.57,'WINGSPAN':35.8,
            'RANGE':3500,'CEILING':39800,'MAX_SPEED':473,'PAX_CAP':195},
    'A320':{'MODEL':'A320','ICAO':'A320','CLASS':'L','WAKE':'M','ENG_TYPE':'J','ENG_NUM':2,
            'ENG_NAME':'IAE V2500/CFM56-5','ENG_MAN':'International Aero Engines/CFM International',
            'ENG_MODEL':'IAE V2500/CFM56-5','ENG_THRUST':120000,'LENGTH':37.57,'WINGSPAN':35.8,
            'TAIL_HEIGHT':11.76,'RANGE':3300,'CEILING':41000,'MAX_SPEED':470,'PAX_CAP':195},
    'A333':{'MODEL':'A330-300','ICAO':'A333','CLASS':'L','WAKE':'H','ENG_TYPE':'J',
            'ENG_NUM':2,'ENG_NAME':'GE CF6-80E/PW4000/RR Trent 700',
            'ENG_MAN':'General Electric/Pratt & Whitney/Rolls-Royce',
            'ENG_MODEL':'CF6-80E/PW4000/Trent 700','ENG_THRUST':316000,'LENGTH':63.67,'WINGSPAN':60.3,
            'TAIL_HEIGHT':17.39,'RANGE':6350,'CEILING':41100,'MAX_SPEED':470,'PAX_CAP':440},
     'B734':{'MODEL':'737-400','ICAO':'B734','CLASS':'L','WAKE':'M','ENG_TYPE':'J',
            'ENG_NUM':2},
    'B738':{'MODEL':'737-800','ICAO':'B738','CLASS':'L','WAKE':'M','ENG_TYPE':'J',
            'ENG_NUM':2},
    'A321':{'MODEL':'A321','ICAO':'A321','CLASS':'L','WAKE':'M','ENG_TYPE':'J','ENG_NUM':2,
            'ENG_NAME':'IAE V2500A5/CFM56-5B','ENG_MAN':'International Aero Engines/CFM International',
            'ENG_MODEL':'IAE V2500A5/CFM56-5B','ENG_THRUST':147000,'LENGTH':44.51,'WINGSPAN':35.8,
            'TAIL_HEIGHT':11.76,'RANGE':3200,'CEILING':41000,'MAX_SPEED':547,'PAX_CAP':236},
    'B739':{'MODEL':'737-900','ICAO':'B739','CLASS':'L','WAKE':'M','ENG_TYPE':'J','ENG_NUM':2},
    'B77W':{'MODEL':'777-300ER','ICAO':'B77W','CLASS':'L','WAKE':'H','ENG_TYPE':'J','ENG_NUM':2},
    'B38M':{'MODEL':'737 MAX 8','ICAO':'B38M','CLASS':'L','WAKE':'M','ENG_TYPE':'J','ENG_NUM':2},
    'B737':{'MODEL':'737-700','ICAO':'B737','CLASS':'L','WAKE':'M','ENG_TYPE':'J','ENG_NUM':2},
    'B772':{'MODEL':'777-200','ICAO':'B772','CLASS':'L','WAKE':'H','ENG_TYPE':'J','ENG_NUM':2},
    'B744':{'MODEL':'747-400 (international, winglets)','ICAO':'B744','CLASS':'L','WAKE':'H',
            'ENG_TYPE':'J','ENG_NUM':4},
    'B763':{'MODEL':'767-300','ICAO':'B763','CLASS':'L','WAKE':'H','ENG_TYPE':'J','ENG_NUM':2},
    'A319':{'MODEL':'A319','ICAO':'A319','CLASS':'L','WAKE':'M','ENG_TYPE':'J','ENG_NUM':2,
            'ENG_NAME':'IAE V2500/CFM56','ENG_MAN':'International Aero Engines/CFM International',
            'ENG_MODEL':'IAE V2500/CFM56','ENG_THRUST':100000,'LENGTH':33.84,'WINGSPAN':35.8,
            'TAIL_HEIGHT':11.76,'RANGE':3750,'CEILING':41000,'MAX_SPEED':547,'PAX_CAP':160},
    'B752':{'MODEL':'757-200','ICAO':'B752','CLASS':'L','WAKE':'M','ENG_TYPE':'J','ENG_NUM':2},
    'MD11':{'MODEL':'MD-11','ICAO':'MD11','CLASS':'L','WAKE':'H','ENG_TYPE':'J','ENG_NUM':3,
           },
    'B77L':{'MODEL':'777-200LR','ICAO':'B77L','CLASS':'L','WAKE':'H','ENG_TYPE':'J','ENG_NUM':2},
    'A306':{'MODEL':'A-300B4-600','ICAO':'A306','CLASS':'L','WAKE':'H','ENG_TYPE':'J',
            'ENG_NUM':2},
    'B39M':{'MODEL':'737 MAX 9','ICAO':'B39M','CLASS':'L','WAKE':'M','ENG_TYPE':'J','ENG_NUM':2},
    'A318':{'MODEL':'A318','ICAO':'A318','CLASS':'L','WAKE':'M','ENG_TYPE':'J','ENG_NUM':2,
            'ENG_NAME':'PW6000A/CFM56-5B','ENG_MAN':'Pratt & Whitney/CFM International',
            'ENG_MODEL':'PW6000A/CFM56-5B','ENG_THRUST':106000,'LENGTH':31.44,'WINGSPAN':34.1,
            'TAIL_HEIGHT':12.56,'RANGE':3100,'CEILING':41000,'MAX_SPEED':470,'PAX_CAP':136},
    'A388':{'MODEL':'A380','ICAO':'A388','CLASS':'L','WAKE':'S','ENG_TYPE':'J','ENG_NUM':4,
            'ENG_NAME':'RR Trent 900/EA GP7000','ENG_MAN':'Rolls-Royce/Engine Alliance',
            'ENG_MODEL':'Trent 900/GP7000','ENG_THRUST':311000,'LENGTH':72.72,'WINGSPAN':79.75,
            'TAIL_HEIGHT':24.09,'RANGE':8200,'CEILING':43000,'MAX_SPEED':551,'PAX_CAP':868},
    'B748':{'MODEL':'747-8','ICAO':'B748','CLASS':'L','WAKE':'H','ENG_TYPE':'J','ENG_NUM':4}}

# Convert the dictionary to a pandas DataFrame for easier feature engineering
aircraft_df = pd.DataFrame.from_dict(aircraft_type_dict, orient='index')

# Rename ICAO column to match flight datasets
aircraft_df = aircraft_df.rename(columns={'ICAO':'aircraft_type'})

In [18]:
# get codes of aircrafts available in the openap library
avaiable_aircraft = prop.available_aircraft()

In [19]:
# ================================================================
# Extract OpenAP Aircraft & Engine Properties as Feature Dictionary
# ---------------------------------------------------------------
# This function retrieves detailed aircraft and engine characteristics
# from the OpenAP library for a given ICAO code. It produces a flattened
# dictionary suitable for feature engineering in fuel prediction.
#
# Parameters:
#   icao_code (str) : ICAO aircraft type code (e.g., 'B744', 'A320')
#
# Returns:
#   dict : Flattened dictionary containing:
#       - Aircraft-level properties (mass, wing, fuselage, drag, etc.)
#       - Engine-level properties, prefixed with 'eng_'
#
# Notes:
#   - Aircraft synonyms are automatically resolved using `use_synonym=True`.
#   - Nested dictionaries are flattened for easier integration into a
#     pandas DataFrame.
#   - The default engine of the aircraft (`engine_default`) is used.
# ================================================================

def get_openap_xtics(icao_code):
    # Retrieve aircraft properties from OpenAP
    prop_dict = prop.aircraft(icao_code, use_synonym=True)
    
    # Flatten nested aircraft property dictionaries
    breakdown_dict = {}
    for k, v in prop_dict.items():
        if isinstance(v, dict):
            for kk, vv in v.items():
                if kk != 'options':  # Exclude optional subfields
                    breakdown_dict[f'{k}_{kk}'] = vv
        else:
            breakdown_dict[k] = v

    # Retrieve and flatten default engine properties
    engine_dict = prop.engine(breakdown_dict['engine_default'])
    for k, v in engine_dict.items():
        breakdown_dict[f'eng_{k}'] = v

    return breakdown_dict


In [20]:
# ================================================================
# Generate OpenAP Aircraft Feature Dictionaries for All Aircraft Types
# ---------------------------------------------------------------
# For each aircraft type in `aircraft_df`, retrieve its OpenAP properties
# using `get_openap_xtics()`. These properties will be used as features
# for fuel consumption modeling.
#
# Logic:
#   - If the lowercase ICAO code is not in `avaiable_aircraft`, it means
#     OpenAP resolved the type using a synonym. Mark `synonym = 1`.
#   - Otherwise, mark `synonym = 0`.
#   - Append the resulting dictionary to `openap_xtics_dict`.
#
# Each dictionary contains:
#   • Flattened aircraft properties
#   • Flattened engine properties (prefixed with 'eng_')
#   • 'aircraft_type' and 'synonym' flags
# ================================================================

openap_xtics_dict = []

for airc_typ in aircraft_df['aircraft_type'].unique():
    xtics = get_openap_xtics(airc_typ)
    xtics['aircraft_type'] = airc_typ
    
    # Flag whether OpenAP used a synonym to resolve this aircraft type
    if airc_typ.lower() not in avaiable_aircraft:
        xtics['synonym'] = 1
    else:
        xtics['synonym'] = 0
    
    openap_xtics_dict.append(xtics)


In [21]:
# convert extracted details from dictionary to dataframe
openap_prop_df = pd.DataFrame(openap_xtics_dict)

In [22]:
# Map each available aircraft in the train,rank and final datasets to the closest aircraft
# available in the acropole model
acropole_synonym = {
    # Airbus A320 Family (Matches within the destination list)
    'A318': 'A318',
    'A319': 'A319',
    'A320': 'A320',
    'A321': 'A321',
    'A20N': 'A320', # A320neo (maps to A320 predecessor)
    'A21N': 'A321', # A321neo (maps to A321 predecessor)

    # Boeing 737 Family (Matches within the destination list)
    'B734': 'B734',
    'B737': 'B737',
    'B738': 'B738',
    'B739': 'B739',
    'B38M': 'B738', # 737 MAX 8 (maps to 737-800 predecessor)
    'B39M': 'B739', # 737 MAX 9 (maps to 737-900 predecessor)

    # Airbus Wide-body (Matches within the destination list)
    'A332': 'A332',
    'A333': 'A333',
    
    # Boeing Wide-body (Matches within the destination list)
    'B763': 'B763',

    # Regional/Misc. Jets (Matches within the destination list)
    'CRJ9': 'CRJ9',
    'E170': 'E170',
    'E190': 'E190',
    'E195': 'E195',
    'E75L': 'E75L',
    'B752': 'B752', # Mid-size narrowbody, no direct match in the list, but B752 is valid

    # Turboprops (Matches within the destination list)
    'AT72': 'AT72',
    'AT75': 'AT75',
    'AT76': 'AT76',

    # Aircraft not in the destination list, matched to closest available alternative:
    'B788': 'A332', # Boeing 787-8 (modern widebody, nearest match in list is A330-200 role)
    'B789': 'A333', # Boeing 787-9 (modern widebody, nearest match in list is A330-300 role)
    'A359': 'A333', # Airbus A350-900 (modern widebody, nearest match in list is A330-300 role)
    'B77W': 'A333', # Boeing 777-300ER (large widebody, nearest match in list is A330-300 role)
    'B772': 'A332', # Boeing 777-200 (widebody, nearest match in list is A330-200 role)
    'B77L': 'A332', # Boeing 777-200LR (widebody, nearest match in list is A330-200 role)
    'A306': 'A332', # Airbus A300-600 (older widebody, nearest match in list is A330-200 role)
    'MD11': 'A333', # McDonnell Douglas MD-11 (widebody, nearest match in list is A330-300 role)
    'B744': 'A333', # Boeing 747-400 (very large widebody, general widebody match)
    'B748': 'A333', # Boeing 747-8 (very large widebody, general widebody match)
    'A388': 'A333', # Airbus A380 (massive widebody, general widebody match)
}


In [23]:
# ================================================================
# Load and Clean FAA Aircraft Data
# ---------------------------------------------------------------
# The FAA Aircraft Characteristics Database provides detailed
# specifications for many aircraft types. This dataset can supplement
# OpenAP data with additional features relevant for fuel modeling.
#
# Source:
#   • FAA Aircraft Characteristics Database
#     https://www.faa.gov/airports/engineering/aircraft_char_database/
#
# Steps:
# 1. Read the 'ACD_Data' sheet from the FAA Excel file.
# 2. Drop columns considered unuseful for our modeling:
#       - 'FAA_Designator', 'Model_FAA', 'Model_BADA'
# 3. Remove the last 20 columns (assumed to be irrelevant metadata).
# 4. Rename the ICAO code column to `aircraft_type` for consistency with
#    other datasets.
# ================================================================

more_aircraft_data = pd.read_excel(FAA_AIRCRAFT_DATA, sheet_name='ACD_Data')

# Drop unnecessary columns
madc_drop_cols = ['FAA_Designator', 'Model_FAA', 'Model_BADA']
more_aircraft_data_cl = more_aircraft_data.iloc[:, :-20].drop(columns=madc_drop_cols)

# Rename ICAO code column for consistency
more_aircraft_data_cl = more_aircraft_data_cl.rename(columns={'ICAO_Code': 'aircraft_type'})


In [24]:
# ================================================================
# Merge Aircraft Data Sources into a Comprehensive Dataset
# ---------------------------------------------------------------
# To create a single, enriched aircraft specification table, we merge:
#   1. `aircraft_df`           : basic aircraft info (model, ICAO, engines)
#   2. `more_aircraft_data_cl` : FAA data with additional technical parameters
#   3. `openap_prop_df`        : OpenAP aircraft properties (drag, thrust, etc.)
#
# Additional processing:
#   - Compute the `wingspan` column:
#       • Use 'Wingspan_ft_without_winglets_sharklets' when available
#       • Otherwise, fall back to 'Wingspan_ft_with_winglets_sharklets'
#
# The resulting `full_aircraft_data` contains comprehensive specifications
# suitable for feature engineering in fuel prediction models.
# ================================================================

# Merge basic aircraft info with cleaned FAA data
full_aircraft_data = pd.merge(
    aircraft_df,
    more_aircraft_data_cl,
    on='aircraft_type',
    how='left'
)

# Fill wingspan using preferred measurement
full_aircraft_data['wingspan'] = np.where(
    full_aircraft_data['Wingspan_ft_without_winglets_sharklets'].isna(),
    full_aircraft_data['Wingspan_ft_with_winglets_sharklets'],
    full_aircraft_data['Wingspan_ft_without_winglets_sharklets']
)

# Merge in OpenAP properties
full_aircraft_data = pd.merge(
    full_aircraft_data,
    openap_prop_df,
    on='aircraft_type',
    how='left'
)


In [25]:
# furthere clean data leaving out data already captured
redundant_cols = ['CLASS','ENG_TYPE','ENG_NUM','LENGTH','WINGSPAN',
                  'Wingspan_ft_without_winglets_sharklets',
                  'Wingspan_ft_with_winglets_sharklets','TAIL_HEIGHT']
full_aircraft_data = full_aircraft_data.drop(columns=redundant_cols)

In [26]:
# ================================================================
# Enrich Flight Lists with Full Aircraft Specifications
# ---------------------------------------------------------------
# Merge the comprehensive aircraft dataset (`full_aircraft_data`) with
# each flight list to attach detailed aircraft characteristics to each
# flight
#
# The merge is performed on `aircraft_type` for all datasets:
#   - Training flights (`train_flightlist`)
#   - Ranking/validation flights (`rank_flightlist`)
#   - Final submission flights (`final_flightlist`)
# ================================================================
train_flightlist = pd.merge(train_flightlist,full_aircraft_data,on='aircraft_type',how='left')
rank_flightlist = pd.merge(rank_flightlist,full_aircraft_data,on='aircraft_type',how='left')
final_flightlist = pd.merge(final_flightlist,full_aircraft_data,on='aircraft_type',how='left')

In [27]:
# ================================================================
# Compute the Haversine Distance Between Two Geographic Points
# ---------------------------------------------------------------
# The Haversine formula calculates the great-circle distance between
# two points on a sphere (Earth) given their latitude and longitude.
# This is useful for estimating straight-line distances between
# airports or waypoints.
#
# Parameters:
#   lat_lon1 : tuple or list of (latitude, longitude) in degrees
#   lat_lon2 : tuple or list of (latitude, longitude) in degrees
#
# Returns:
#   float : distance between the two points in meters
#
# Notes:
#   - The Earth's radius is assumed to be 6371 km.
#   - Inputs are converted from degrees to radians before computation.
# ================================================================

def haversine(lat_lon1, lat_lon2):
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert latitude and longitude from degrees to radians
    lat1, lon1 = map(np.radians, lat_lon1)
    lat2, lon2 = map(np.radians, lat_lon2)
    
    # Haversine formula components
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    # Distance in meters
    return R * c * 1000


In [28]:
# ================================================================
# Compute Total Duration per Flight Phase
# ---------------------------------------------------------------
# This function calculates the total time spent in each flight phase
# (e.g., takeoff, climb, cruise, descent, landing) for a given flight
# DataFrame.
#
# Parameters:
#   ndf   : pandas.DataFrame
#           Flight-level time series containing at least:
#             - 'timestamp' column (datetime)
#             - Column specifying the flight phase (categorical)
#   phase : str
#           Name of the column representing flight phases
#
# Returns:
#   dict : mapping of {phase_category : total_duration_in_seconds}
#
# Methodology:
#   1. Identify contiguous blocks of the same flight phase.
#   2. For each block, calculate duration as the difference between
#      last and first timestamp.
#   3. Aggregate durations per phase category.
# ================================================================

def get_phase_duration(ndf, phase):
    # Make a copy to avoid modifying the original DataFrame
    df = ndf.copy()
    
    # Identify contiguous groups where the phase value changes
    df['group'] = (df[phase] != df[phase].shift()).cumsum()
    
    # Compute start and end timestamps per group
    group_durations = (
        df.groupby([phase, 'group'])['timestamp']
          .agg(['first', 'last'])
    )
    
    # Compute duration (in seconds) per group
    group_durations['duration'] = (group_durations['last'] - group_durations['first']).dt.total_seconds()
    
    # Sum durations across all groups for each phase
    result = group_durations.groupby(phase)['duration'].sum().to_dict()
    
    return result


In [29]:
# ================================================================
# Retrieve Flight Metadata by Flight ID
# ---------------------------------------------------------------
# This function searches for a flight's metadata across all flight
# datasets (training, ranking/validation, and final submission)
# and returns the corresponding row(s) as a DataFrame.
#
# Parameters:
#   flight_id : str
#       Unique identifier of the flight to retrieve.
#
# Returns:
#   pandas.DataFrame : flight metadata including:
#       - origin and destination airports
#       - takeoff and landing times
#       - aircraft type and associated features
#       - airport coordinates and elevations
#
# Notes:
#   - Searches datasets in the order: train_flightlist → rank_flightlist → final_flightlist
#   - Returns an empty DataFrame if the flight ID is not found.
# ================================================================

def get_flight_metadata(flight_id):
    # Attempt to locate the flight in the training dataset
    fl = train_flightlist[train_flightlist.flight_id == flight_id]
    
    # If not found, search in the ranking/validation dataset
    if fl.empty:
        fl = rank_flightlist[rank_flightlist.flight_id == flight_id]
    
    # If still not found, search in the final submission dataset
    if fl.empty:
        fl = final_flightlist[final_flightlist.flight_id == flight_id]
    
    return fl


In [30]:
# ================================================================
# Evaluate Resampled Flight Data Quality Against Raw Observations
# ---------------------------------------------------------------
# This function computes metrics to assess how closely a resampled
# flight trajectory matches the original raw timestamps.
#
# Parameters:
#   raw_df      : pandas.DataFrame
#                 Original flight trajectory with at least a 'timestamp' column
#   resample_df : pandas.DataFrame
#                 Resampled trajectory to be evaluated, must also contain 'timestamp'
#
# Returns:
#   pandas.DataFrame : `resample_df` enriched with the following columns:
#       - sec_from_real : distance (seconds) to the nearest raw timestamp
#       - raw_diff_mean : mean interval of raw timestamps
#       - raw_diff_count: number of intervals in raw timestamps
#       - raw_diff_max  : maximum interval in raw timestamps
#       - raw_diff_min  : minimum interval in raw timestamps
#       - raw_diff_std  : standard deviation of intervals
#       - auth_score    : exponential decay score reflecting proximity to real timestamps
#
# Methodology:
#   1. Convert timestamps to UTC datetime.
#   2. Compute intervals between consecutive raw timestamps.
#   3. For each resampled timestamp, calculate the distance in seconds
#      to the nearest raw timestamp.
#   4. Compute summary statistics of raw timestamp intervals.
#   5. Compute an "authenticity score" using exponential decay with
#      a half-life of 60 seconds (closer timestamps get higher score).
# ================================================================

def compute_resample_quality(raw_df, resample_df):
    # Ensure timestamps are in datetime format with UTC
    raw_df['timestamp'] = pd.to_datetime(raw_df['timestamp'], utc=True)
    resample_df['timestamp'] = pd.to_datetime(resample_df['timestamp'], utc=True)

    # Compute intervals between consecutive raw timestamps (in seconds)
    raw_interval = raw_df['timestamp'].diff().dt.total_seconds()

    # Convert timestamps to integer seconds since epoch
    real_times = raw_df['timestamp'].astype("int64").to_numpy() // 1_000_000_000
    resampled_times = resample_df['timestamp'].astype("int64").to_numpy() // 1_000_000_000

    # Compute distance to nearest raw timestamp for each resampled point
    dist = np.abs(resampled_times[:, None] - real_times[None, :]).min(axis=1)
    resample_df['sec_from_real'] = dist

    # Attach summary statistics of raw intervals
    resample_df['raw_diff_mean'] = np.mean(raw_interval)
    resample_df['raw_diff_count'] = np.size(raw_interval)
    resample_df['raw_diff_max'] = np.max(raw_interval)
    resample_df['raw_diff_min'] = np.min(raw_interval)
    resample_df['raw_diff_std'] = np.std(raw_interval)

    # Compute authenticity score based on exponential decay
    lam = 1 / 60  # half-life of 60 seconds
    resample_df['auth_score'] = np.exp(-lam * resample_df['sec_from_real'])

    return resample_df


In [31]:
# ================================================================
# Smooth ATM Phase Classification for a Flight
# ---------------------------------------------------------------
# Assigns ATM (Air Traffic Management) phases to flight trajectory
# points based on distance from departure and arrival airports.
#
# Rules:
#   - All points before the last 'DEP' phase are set to 'DEP'
#   - All points after the first 'ARR' phase are set to 'ARR'
#   - Remaining points default to 'ENR' (Enroute)
#
# Parameters:
#   flight_df : pandas.DataFrame
#       Flight trajectory data, must include columns:
#         - 'timestamp'       : datetime of observation
#         - 'dist_from_adep'  : distance to departure airport (NM or km)
#         - 'dist_from_ades'  : distance to destination airport (NM or km)
#
# Returns:
#   pandas.DataFrame : sorted flight_df with smoothed 'atm_phase' column
#
# Notes:
#   - The function sorts by timestamp internally to ensure chronological ordering.
#   - Phases are first assigned based on distance thresholds:
#       • <40 units from departure airport → 'DEP'
#       • <100 units from arrival airport → 'ARR'
#   - Then smoothing rules are applied to ensure logical continuity.
# ================================================================

def get_atm_phases(flight_df):
    # Initialize all phases as Enroute (ENR)
    flight_df['atm_phase'] = 'ENR'
    
    # Assign preliminary DEP and ARR phases based on distance thresholds
    flight_df.loc[flight_df['dist_from_adep'] < 40, 'atm_phase'] = 'DEP'
    flight_df.loc[flight_df['dist_from_ades'] < 100, 'atm_phase'] = 'ARR'

    # Ensure correct chronological order
    df = flight_df.sort_values('timestamp', ignore_index=True).copy()
    
    # Find indices of last DEP and first ARR
    dep_indices = df.index[df['atm_phase'] == 'DEP']
    arr_indices = df.index[df['atm_phase'] == 'ARR']

    last_dep_idx = dep_indices.max() if len(dep_indices) > 0 else None
    first_arr_idx = arr_indices.min() if len(arr_indices) > 0 else None

    # Apply smoothing rules to enforce phase continuity
    if last_dep_idx is not None:
        df.loc[:last_dep_idx, 'atm_phase'] = 'DEP'

    if first_arr_idx is not None:
        df.loc[first_arr_idx:, 'atm_phase'] = 'ARR'

    return df


In [32]:
def prepare_flight_segments_final(train_fuel, flights_dir=TRAIN_FLIGHTS_DIR,
                                  ts_cols=None):
    if ts_cols is None:
        ts_cols = [
            'longitude','altitude','groundspeed',
            'mach','CAS','fuel_flow','fuel',
            'drag','thrust','auth_score','cl_fuel',
            'enr_fuel','dist_from_ades','acp_fuel','acp_fuelflow',
            ]

    phases = ['GND','CL','DE','LVL','CR','NA']
    atm_phases = ['DEP','ENR','ARR']
    all_segments = []

    # --- loop once per flight
    for flight_id, flight_rows in train_fuel.groupby('flight_id'):
        
        #prc776443417 and prc776502511 appear to be parts of the same flight
        if flight_id == 'prc776443417':
            print(flight_id)
            raw_flight_df_1 = pd.read_parquet(TRAIN_FLIGHTS_DIR / "prc776443417.parquet")
            raw_flight_df_2 = pd.read_parquet(TRAIN_FLIGHTS_DIR / "prc776502511.parquet")
            raw_flight_df = pd.concat([raw_flight_df_1,raw_flight_df_2])
            raw_flight_df['flight_id'] = 'prc776443417'

        # prc778592434, prc778582087, and prc778566666 also appear to be parts of the same flight
        elif flight_id == 'prc778592434':
            print(flight_id)
            raw_flight_df_1 = pd.read_parquet(TRAIN_FLIGHTS_DIR / "prc778592434.parquet")
            raw_flight_df_2 = pd.read_parquet(TRAIN_FLIGHTS_DIR / "prc778582087.parquet")
            raw_flight_df_3 = pd.read_parquet(TRAIN_FLIGHTS_DIR / "prc778566666.parquet")
            raw_flight_df = pd.concat([raw_flight_df_1,raw_flight_df_2,raw_flight_df_3])
            raw_flight_df['flight_id'] = 'prc778592434'
        
        
        else:
            try:
                raw_flight_df = pd.read_parquet(flights_dir / f"{flight_id}.parquet")
            except:
                # some flights id in the final_flightlist are available in the rank trajectories dir only
                raw_flight_df = pd.read_parquet(RANK_FLIGHTS_DIR / f"{flight_id}.parquet")
            
        
        flight_data = get_flight_metadata(flight_id)
        
        
        takeoff = pd.to_datetime(flight_data['takeoff'].item(),utc=True)
        landed = pd.to_datetime(flight_data['landed'].item(),utc=True)
        orig_lat = flight_data['orig_lat'].item()
        orig_lon = flight_data['orig_long'].item()
        dest_lat = flight_data['dest_lat'].item()
        dest_lon = flight_data['dest_long'].item()   
        orig_elev = flight_data['orig_elev'].item() 
        dest_elev = flight_data['dest_elev'].item()

        takeoff_minute = pd.to_datetime(takeoff.floor('min'),utc=True)
        landed_minute = pd.to_datetime(landed.ceil('min'),utc=True)

        takeoff_data = flight_data[['orig_elev','orig_long',
                                    'orig_lat','takeoff','aircraft_type',
                                    'flight_id']].copy()
        

        if raw_flight_df['timestamp'].dt.tz is None:
            raw_flight_df['timestamp'] = raw_flight_df['timestamp'].dt.tz_localize('UTC')
        else:
            raw_flight_df['timestamp'] = raw_flight_df['timestamp'].dt.tz_convert('UTC')
        takeoff = pd.to_datetime(takeoff).tz_convert('UTC') if takeoff.tzinfo else takeoff.tz_localize('UTC')
                
        raw_flight_df = raw_flight_df.sort_values('timestamp').reset_index(drop=True)

        #ensure all timestamps for which prediction is required are included
        earliest_predict = pd.to_datetime(flight_rows['start'],utc=True).min().floor('min')
        latest_predict = pd.to_datetime(flight_rows['end'],utc=True).max().ceil('min')

        earliest_raw_flight_time = raw_flight_df['timestamp'].min()
        latest_raw_flight_time = raw_flight_df['timestamp'].max()

        takeoff_data = takeoff_data.rename(columns={'orig_elev':'altitude',
                                                   'orig_long':'longitude',
                                                   'orig_lat':'latitude',
                                                   'takeoff':'timestamp',
                                                   'aircraft_type':'typecode'})
        

        if (earliest_raw_flight_time<takeoff_minute) and (earliest_predict<earliest_raw_flight_time):
            #print('prediction is required for unvavailable period with earliest available being flight trajectory data')
            #add earliest predict before earliest rawflight
            earliest_rawflight = raw_flight_df[raw_flight_df['timestamp']==earliest_raw_flight_time]
            earliest_pred_data = earliest_rawflight.copy()
            earliest_pred_data['timestamp'] = earliest_predict
            raw_flight_df = pd.concat([earliest_pred_data,raw_flight_df])
            #print('just added the required segment')
            #display(raw_flight_df)

        

        elif (earliest_raw_flight_time>=takeoff_minute) and (earliest_predict<takeoff_minute):
            #print('prediction is required for period with earliest available being takeoff')
            #add earliest predict before takeoff data
            
            earliest_pred_data = takeoff_data.copy()
            earliest_pred_data['timestamp'] = earliest_predict
            raw_flight_df = pd.concat([earliest_pred_data,raw_flight_df])
            #print('just added the required segment')
            #display(raw_flight_df)



        
        # if next observation in trajectory after takeoff happens within 2 minutes of takeoff,
        # use the next observation's vertical rate for the takeoff vertical rate
        #vertical rate value in trajectory after takeoff
        obs_after_takeoff = raw_flight_df.loc[raw_flight_df['timestamp']>takeoff].head(1)
        obs_after_takeoff_time = obs_after_takeoff['timestamp'].item()
        time_within_takeoff = (obs_after_takeoff_time - takeoff).total_seconds()/60
        if time_within_takeoff<2:
            obs_after_takeoff_roc = obs_after_takeoff['vertical_rate'].item()
            new_next_altitude = (obs_after_takeoff_roc*time_within_takeoff) + orig_elev
            raw_flight_df.loc[raw_flight_df['timestamp']==obs_after_takeoff_time,'altitude']=new_next_altitude
            takeoff_data['vertical_rate'] = obs_after_takeoff_roc

        pre_append_df = raw_flight_df.copy()
        
        filtered_df = pre_append_df[pre_append_df['timestamp']<=takeoff].copy()

        
        #if there are observations preceding takeoff
        if filtered_df.shape[0]>0:
            
            
            raw_flight_df = pre_append_df

            
            raw_flight_df.loc[raw_flight_df['timestamp']<takeoff,'vertical_rate'] = 0
            raw_flight_df.loc[raw_flight_df['timestamp']<takeoff,'altitude'] = orig_elev

            pre_append_df = raw_flight_df.copy()
        
        land_data = flight_data[['dest_elev','dest_long','dest_lat',
                                 'landed','aircraft_type','flight_id']].copy()
        
        
        land_data = land_data.rename(columns={'dest_elev':'altitude',
                                              'dest_long':'longitude',
                                              'dest_lat':'latitude',
                                              'landed':'timestamp',
                                              'aircraft_type':'typecode',
                                             'vertical_rate':0})
        
        if (latest_raw_flight_time>landed_minute) and (latest_predict>latest_raw_flight_time):
            #add latest predict after latest rawflight
            #print('prediction is required for unvavailable period with latest available being flight trajectory data')
            latest_rawflight = raw_flight_df[raw_flight_df['timestamp']==latest_raw_flight_time]
            latest_pred_data = latest_rawflight.copy()
            latest_pred_data['timestamp'] = latest_predict
            raw_flight_df = pd.concat([raw_flight_df,latest_pred_data])
            #if raw_flight_df['timestamp'].max()< latest_predict:
            #    print('why latest predict still absent')

        
        elif (latest_raw_flight_time<=landed_minute) and (latest_predict>landed_minute):
            #print('prediction is required for unavailable period with latest available being landed data')
            #add latest predict after landed data
        
            latest_pred_data = land_data.copy()
            latest_pred_data['timestamp'] = latest_predict
            raw_flight_df = pd.concat([raw_flight_df,latest_pred_data])
            #if raw_flight_df['timestamp'].max()< latest_predict:
            #    print('why latest predict still absent')
            
        
        pre_append_df = raw_flight_df.copy()

        lfiltered_df = pre_append_df[pre_append_df['timestamp']>=landed].copy()
        if lfiltered_df.shape[0]>0:
            
            raw_flight_df = pre_append_df

            
            raw_flight_df.loc[raw_flight_df['timestamp']>landed,'vertical_rate'] = 0
            raw_flight_df.loc[raw_flight_df['timestamp']>landed,'altitude'] = dest_elev
            
            
            pre_append_df = raw_flight_df.copy()
        
        
        land_data['timestamp'] = pd.to_datetime(land_data['timestamp'],utc=True)
        takeoff_data['timestamp'] = pd.to_datetime(takeoff_data['timestamp'],utc=True)
        raw_flight_df = pd.concat([takeoff_data,raw_flight_df,land_data],axis=0).sort_values('timestamp').reset_index(drop=True)
            

        
        res_flight_df = Traffic(raw_flight_df).filter().resample('60s').eval().data
        
        res_flight_df.loc[res_flight_df['timestamp']<takeoff_minute,'vertical_rate'] = 0
        res_flight_df.loc[res_flight_df['timestamp']>landed_minute,'vertical_rate'] = 0

        res_flight_df.loc[res_flight_df['timestamp']<=takeoff_minute,'altitude'] = orig_elev
        res_flight_df.loc[res_flight_df['timestamp']>=landed_minute,'altitude'] = dest_elev


        
        flight_df = compute_resample_quality(raw_flight_df.copy(),res_flight_df)
        fl_max = flight_df['altitude'].max()
        
        
            
        flight_df = flight_df.sort_values('timestamp').reset_index(drop=True)

        #get NM distance from departing airport
        flight_df['dist_from_adep'] = flight_df.apply(
            lambda r: haversine((r.latitude,r.longitude),
                                (orig_lat,orig_lon)),axis=1)
        flight_df['dist_from_ades'] = flight_df.apply(
            lambda r: haversine((r.latitude,r.longitude),
                                (dest_lat,dest_lon)),axis=1)
        flight_df['dist_from_ades'] = flight_df['dist_from_ades']/1852
        flight_df['dist_from_adep'] = flight_df['dist_from_adep']/1852

        flight_df = get_atm_phases(flight_df)
        
        flight_start = flight_df['timestamp'].min()
        flight_end = flight_df['timestamp'].max()
        
        # Precompute arrays
        ts = (flight_df.timestamp - flight_start).dt.total_seconds().to_numpy()
        alt = flight_df['altitude'].to_numpy()
        
        times = flight_df['timestamp'].to_numpy()
        tas = flight_df['TAS'].fillna(flight_df['CAS']).fillna(flight_df['groundspeed']).to_numpy()
        roc = flight_df['vertical_rate'].to_numpy()

        typecode = flight_df['typecode'].dropna().unique()[0]
        fe = FuelEstimator()
        

        acpe_typecode = acropole_synonym[typecode.upper()]
        acpe_flight = flight_df.copy()
        acpe_flight['ts'] = ts
        acpe_flight['tas_filled'] = tas
        acpe_flight['type_mapped'] = acpe_typecode
        flight_fuel = fe.estimate(
            acpe_flight,typecode="type_mapped",groundspeed="groundspeed",
            altitude="altitude",vertical_rate="vertical_rate",second="ts",
            airspeed="tas_filled",)

        flight_fuel['inst_fuel'] = flight_fuel['fuel_cumsum'].diff()
        flight_fuel['inst_fuel'] = flight_fuel['inst_fuel'].fillna(flight_fuel['fuel_cumsum'].iloc[0])

        flight_df['acp_fuel'] = flight_fuel['inst_fuel']
        flight_df['acp_fuelflow'] = flight_fuel['fuel_flow']
        
        acp_fuelflow = flight_df['acp_fuelflow'].to_numpy()

        
        # --- Phase labeling
        fp = FlightPhase()
        fp.set_trajectory(ts, alt, tas, roc)
        flight_df['phase'] = fp.phaselabel()

        #enforce ground phases
        flight_df.loc[flight_df['timestamp']>landed_minute,'phase'] = 'GND'
        flight_df.loc[flight_df['timestamp']<takeoff_minute,'phase'] = 'GND'
              
               
        fphases = flight_df['phase'].to_numpy()
        atmphases = flight_df['atm_phase'].to_numpy()

        # Phase counts and durations (with defaults)
        flight_phase_counts = flight_df['phase'].value_counts(normalize=True).to_dict()
        
        
        for k in phases:
            flight_phase_counts.setdefault(k, 0)
            
        flight_phase_counts = {f'flight_{k}_ct': v for k, v in flight_phase_counts.items()}
        
        
        # --- Fuel flow computation (vectorized per-row loop)
        mtow = openap_prop_df.loc[
            openap_prop_df['aircraft_type'] == typecode, 'mtow'
        ].item()

        
        fuelflow = FuelFlow(typecode, use_synonym=True)
        dragobj = Drag(ac=typecode,use_synonym=True)
        thrustobj = Thrust(ac=typecode,use_synonym=True)
        
        
        # Compute time deltas (seconds)
        
        d_ts = flight_df['timestamp'].diff().dt.total_seconds().bfill()

        fuelflow_every_step = []
        fuel_every_step = []
        drag_every_step = []
        thrust_every_step = []
        
        mass_current = mtow
        for i in range(flight_df.shape[0]):
            drag = dragobj.clean(mass=mass_current, tas=tas[i],
                                 alt=alt[i], vs=roc[i])
            if fphases[i] in ['CR']:
                thrust = thrustobj.cruise(alt=alt[i],tas=tas[i])
            elif fphases[i] in ['LVL','NA']:
                if (atmphases[i]=='DEP') and (times[i]>takeoff):
                    thrust = thrustobj.climb(alt=alt[i],tas=tas[i],roc=roc[i])
                elif atmphases[i]=='ENR':
                    thrust = thrustobj.cruise(alt=alt[i],tas=tas[i])
                elif (atmphases[i]=='ARR') and (times[i]<landed):
                    thrust = thrustobj.descent_idle(tas=tas[i], alt=alt[i])
                else:
                    thrust = np.nan
            elif fphases[i]=='CL':
                thrust = thrustobj.climb(alt=alt[i],tas=tas[i],roc=roc[i])
            elif fphases[i]=='DE':
                thrust = thrustobj.descent_idle(tas=tas[i], alt=alt[i])
            else:
                thrust = np.nan

            
            drag_every_step.append(drag)
            thrust_every_step.append(thrust)
            ff = fuelflow.enroute(
                mass=mass_current,
                tas=tas[i],
                alt=alt[i],
                vs=roc[i],
                )
            if np.isnan(ff):
                ff = acp_fuelflow[i]

            if np.isnan(ff):
                candidates = [
                    v for v, p, a in zip(fuelflow_every_step[:i], fphases[:i], alt[:i])
                    if (not np.isnan(v)) and (p == fphases[i]) and abs(a-alt[i]) < 1500
                ]
                if candidates:
                    ff = np.nanmedian(candidates)
                candidates2 = [
                    v for v, p in zip(fuelflow_every_step[:i], fphases[:i])
                    if (not np.isnan(v)) and (p == fphases[i])
                ]
                if candidates2 and np.isnan(ff):
                    ff = np.nanmedian(candidates2)
                    
                if np.isnan(ff):
                    ff = np.nanmedian(fuelflow_every_step)
            
            fuel = ff * d_ts[i]
            
            
            fuelflow_every_step.append(ff)
            fuel_every_step.append(fuel)
            if np.isnan(fuel):
                pass
            else:
                mass_current -= fuel

        atmphase_lists = {p: [] for p in ['DEP','ENR','ARR']}
        
        phase_lists = {p: [] for p in ['CR','CL','LVL','DE','NA','GND']}

        for ph, fuel in zip(fphases, fuel_every_step):
            for p in phase_lists:
                phase_lists[p].append(fuel if ph == p else 0)

        cr_fuel  = phase_lists['CR']
        cl_fuel  = phase_lists['CL']
        lvl_fuel = phase_lists['LVL']
        de_fuel  = phase_lists['DE']
        na_fuel  = phase_lists['NA']
        gnd_fuel = phase_lists['GND']

        for aph, fuel in zip(atmphases, fuel_every_step):
            for p in atmphase_lists:
                atmphase_lists[p].append(fuel if aph == p else 0)        
        
        dep_fuel  = atmphase_lists['DEP']
        enr_fuel  = atmphase_lists['ENR']
        arr_fuel = atmphase_lists['ARR']
        
        flight_df = flight_df.assign(fuel_flow=fuelflow_every_step,cl_fuel=cl_fuel,
                                     fuel=fuel_every_step,gnd_fuel=gnd_fuel,
                                    drag = drag_every_step,cr_fuel=cr_fuel,
                                    thrust = thrust_every_step,lvl_fuel=lvl_fuel,
                                    na_fuel=na_fuel,de_fuel=de_fuel,
                                    dep_fuel=dep_fuel,enr_fuel=enr_fuel,
                                     arr_fuel=arr_fuel,d_ts=d_ts)
        
        
        # fill gnd thrust with minimum idle thrust
        flight_df.loc[
            (flight_df['thrust'].isna()) & (flight_df['phase'] == 'GND'),
            'thrust'
        ] = flight_df.loc[flight_df['phase'] == 'DE', 'thrust'].min() / 2
        
        # fill gnd fuelflow with minimum fuelflow
        flight_df.loc[
            (flight_df['fuel_flow'].isna()) & (flight_df['phase'] == 'GND'),
            'fuel_flow'
        ] = flight_df.loc[flight_df['phase'] == 'DE', 'fuel_flow'].min() / 2

        #fill na thrust at cl and departure with max thrust
        flight_df.loc[
            (flight_df['thrust'].isna()) & (flight_df['phase'] == 'CL') & (flight_df['atm_phase']=='DEP'),
            'thrust'
        ] = flight_df.loc[flight_df['phase'] == 'CL', 'thrust'].max()

        #fill na fuelflow at CL and DEP phase with max fuel flow
        flight_df.loc[
            (flight_df['fuel_flow'].isna()) & (flight_df['phase'] == 'CL') & (flight_df['atm_phase']=='DEP'),
            'fuel_flow'
        ] = flight_df.loc[flight_df['phase'] == 'CL', 'fuel_flow'].max()
        
        

        #fill gnd fuel fom assumed fuel_flow
        flight_df['fuel'] = flight_df['fuel_flow'] * flight_df['d_ts']

        flight_df.drop(columns=['d_ts'],inplace=True)
        
        total_flight_fuel = np.nan if flight_df['fuel'].sum()==0 else flight_df['fuel'].sum()
        
        
        ts_array = flight_df['timestamp'].to_numpy()
        raw_ts = raw_flight_df['timestamp'].to_numpy()
        
        # --- Process each fuel segment
        for _, row in flight_rows.iterrows():
            start, end = pd.to_datetime(row['start'],
                                        utc=True), pd.to_datetime(row['end'],
                                                                  utc=True)
            
            mask = (ts_array >= start) & (ts_array <= end)
            seg = flight_df.loc[mask]
            raw_mask = (raw_ts >= start) & (raw_ts <= end)
            raw_seg_df = raw_flight_df[raw_mask]

            #some segments needed for prediction turned out to require for less than 1 minute:
            if (start-end).total_seconds()<60 and seg.empty:
                mask = (ts_array >= start) & (ts_array <= end.ceil('min'))
                seg = flight_df.loc[mask]
                raw_mask = (raw_ts >= start) & (raw_ts <= end.ceil('min'))
                raw_seg_df = raw_flight_df[raw_mask]

                if seg.empty:
                    mask = (ts_array >= start.floor('min')) & (ts_array <= end)
                    seg = flight_df.loc[mask]
                    raw_mask = (raw_ts >= start.floor('min')) & (raw_ts <= end)
                    raw_seg_df = raw_flight_df[raw_mask]
            

            
            
            raw_seg_interval = raw_seg_df['timestamp'].diff().dt.total_seconds()

            seg_scalars = {
                'seg_start': start,
                'seg_end': end,
                'flight_id': flight_id,
                'fuel_kg': row['fuel_kg'],
                'flight_fuel':total_flight_fuel,
                'flight_start':flight_start,
                'flight_end': flight_end,
                'real_flight_dur':(flight_end-flight_start).total_seconds(),
                'fl_max_alti': fl_max
            }
            
            seg_scalars['rawseg_diff_count'] = np.size(raw_seg_interval)
            seg_scalars['rawseg_diff_mean'] = np.mean(raw_seg_interval) if seg_scalars['rawseg_diff_count']>0 else np.nan
            
            

            if seg.empty:
                seg_scalars['missing_segment'] = 1
            else:
                seg_scalars['missing_segment'] = 0

            # Segment phase counts/durations
            seg_phase_durations = get_phase_duration(seg,'phase')
            seg_atmphase_durations = get_phase_duration(seg,'atm_phase')
            
            for k in phases:
                seg_phase_durations.setdefault(k, 0)
            seg_phase_durations = {f'seg_{k}_dur': v for k, v in seg_phase_durations.items()}
            
            for k in atm_phases:
                seg_atmphase_durations.setdefault(k, 0)
            seg_atmphase_durations = {f'seg_{k}_dur': v for k, v in seg_atmphase_durations.items()}

            seg_scalars.update(seg_phase_durations)
            seg_scalars.update(flight_phase_counts)
            seg_scalars.update(seg_atmphase_durations)

            
            seg_scalars['all_seg_phase_dur'] = sum(seg_phase_durations.values())
            seg_dur = (end - start).total_seconds()
            
            seg_scalars['unscaled_approx_seg_fuel'] = (seg_dur/seg_scalars['real_flight_dur'])*total_flight_fuel
            

            # --- Column stats (NumPy vectorized)
            for col in ts_cols:
                #print(col)
                arr = seg[col].to_numpy()
                valid = ~np.isnan(arr)
                if not valid.any():
                    for stat in ['start','end','mean','min','max',
                                 'sum','nancount']:
                        seg_scalars[f'{stat}_{col}'] = np.nan
                    seg_scalars[f'nancount_{col}'] = arr.size
                    continue

                data = arr[valid]
                if col in ['longitude','thrust','acp_fuel']:
                    seg_scalars[f'start_{col}'] = data[0]
                if col in ['altitude','auth_score']:
                    seg_scalars[f'end_{col}'] = data[-1]
                if col in ['altitude']:
                    seg_scalars[f'mean_{col}'] = data.mean()
                if col in ['longitude']:
                    seg_scalars[f'min_{col}'] = data.min()
                if col in ['acp_fuel']:
                    seg_scalars[f'max_{col}'] = data.max()
                if col in ['altitude','groundspeed','CAS','fuel_flow','fuel',
                           'drag','thrust','cl_fuel','enr_fuel','dist_from_ades',
                           'acp_fuel','acp_fuelflow']:
                    seg_scalars[f'sum_{col}'] = data.sum()
                if col in ['mach']:
                    seg_scalars[f'nancount_{col}'] = arr.size - data.size

                if col == 'altitude' and data.size > 1:
                    diff = np.diff(data)
                    seg_scalars['total_climb_height'] = diff[diff > 0].sum()

            all_segments.append(seg_scalars)

    return pd.DataFrame(all_segments)

In [ ]:
prepared_train = prepare_flight_segments_final(train_fuel,TRAIN_FLIGHTS_DIR)

In [36]:
prep_train = pd.merge(prepared_train,train_flightlist,on='flight_id',how='left')

In [ ]:
prep_train.to_csv(f'prep_train_acropole.csv',index=False)

In [ ]:
prepared_rank = prepare_flight_segments_final(rank_fuel,RANK_FLIGHTS_DIR)

In [106]:
prep_rank = pd.merge(prepared_rank,rank_flightlist,on='flight_id',how='left')

In [107]:
prep_rank.to_csv(f'prep_rank_acropole.csv',index=False)

In [ ]:
prepared_final = prepare_flight_segments_final(final_fuel,FINAL_FLIGHTS_DIR)

In [136]:
prep_final = pd.merge(prepared_final,final_flightlist,on='flight_id',how='left')

In [137]:
prep_final.to_csv(f'prep_final_acropole.csv',index=False)